In [1]:
# Install required libraries
!pip install langchain pandas faiss-cpu torch --quiet
!pip install langchain-community sentence-transformers --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.

In [2]:
import pandas as pd
import os
import re
from tqdm import tqdm
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import torch
import shutil

In [4]:
FUNDAMENTAL_CSV_PATH = '/content/fundamental_data.csv'
OUTPUT_DIR = '/content/fundamental_data_vector_db'

# Create the output directory if it doesn’t exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Step 1: Load the CSV file and inspect it
df = pd.read_csv(FUNDAMENTAL_CSV_PATH)
print("First 5 rows of the dataset:")
print(df.head(5))

First 5 rows of the dataset:
  Company Symbol                                     Company Name  \
0          SIGS3  Siddhartha Investment Growth Scheme 3\t (SIGS3)   
1           NWCL         Nepal Warehousing Company Limited (NWCL)   
2           SPIL      Siddhartha Premier Insurance Limited (SPIL)   
3            SBL                    Siddhartha Bank Limited (SBL)   
4         GWFD83     12% Goodwill finance Debenture 2083 (GWFD83)   

                Sector Shares Outstanding                       EPS P/E Ratio  \
0          Mutual Fund      80,580,000.00   0.00 (FY:081-082, Q:10)         0   
1               Others       6,875,000.00  -13.06 (FY:081-082, Q:1)    -75.04   
2   Non-Life Insurance      28,065,499.00   16.03 (FY:081-082, Q:2)      54.1   
3     Commercial Banks     140,899,801.90    7.53 (FY:081-082, Q:2)     38.11   
4  Corporate Debenture                NaN           0.00 (FY:, Q:0)         0   

  Book Value       PBV Market Capitalization  \
0          0         

In [5]:
# Drop non-fundamental columns
df = df.drop(columns=['% Change', 'Last Traded On', 'Shares Outstanding','Market Price','% Bonus','Right Share','% Dividend'], errors='ignore')

In [6]:
df.head(5)

,Company Symbol,Company Name,Sector,EPS,P/E Ratio,Book Value,PBV,Market Capitalization
0,SIGS3,Siddhartha Investment Growth Scheme 3\t (SIGS3),Mutual Fund,"0.00 (FY:081-082, Q:10)",0,0,0,"797,742,000.00"
1,NWCL,Nepal Warehousing Company Limited (NWCL),Others,"-13.06 (FY:081-082, Q:1)",-75.04,88.89,11.02,"6,737,500,000.00"
2,SPIL,Siddhartha Premier Insurance Limited (SPIL),Non-Life Insurance,"16.03 (FY:081-082, Q:2)",54.1,287.34,3.02,"24,341,207,282.70"
3,SBL,Siddhartha Bank Limited (SBL),Commercial Banks,"7.53 (FY:081-082, Q:2)",38.11,198.61,1.45,"40,438,243,145.30"
4,GWFD83,12% Goodwill finance Debenture 2083 (GWFD83),Corporate Debenture,"0.00 (FY:, Q:0)",0,0,"1,205.40",0


In [8]:
print("\nMissing values before filling:")
print(df.isnull().sum())


Missing values before filling:
Company Symbol           0
Company Name             0
Sector                   0
EPS                      0
P/E Ratio                0
Book Value               0
PBV                      0
Market Capitalization    0
dtype: int64


In [9]:
# Step 3: Standardize Company Names (remove extra spaces)
df["Company Name"] = df["Company Name"].apply(
    lambda x: re.sub(r'\s+', ' ', x.strip())  # Remove extra spaces
)


In [10]:
# Convert DataFrame to list of dictionaries
companies = df.to_dict('records')
print("\nSample preprocessed company entry:")
print(companies[0])


Sample preprocessed company entry:
{'Company Symbol': 'SIGS3', 'Company Name': 'Siddhartha Investment Growth Scheme 3 (SIGS3)', 'Sector': 'Mutual Fund', 'EPS': '0.00 (FY:081-082, Q:10)', 'P/E Ratio': '0', 'Book Value': '0', 'PBV': '0', 'Market Capitalization': '797,742,000.00'}


In [11]:
# Step 4: Convert Company Data to Text Chunks
chunks = []
metadata = []

for company in tqdm(companies, desc="Creating Company Chunks"):
    # Build the text chunk
    chunk_parts = [f"{company['Company Name']} ({company['Company Symbol']}), a {company['Sector']} company"]

    # Include EPS unconditionally since no missing values
    chunk_parts.append(f"with an EPS of {company['EPS']}")

    # Include P/E Ratio if non-zero, handling commas
    if float(str(company["P/E Ratio"]).replace(",", "")) != 0:
        chunk_parts.append(f"and a P/E ratio of {company['P/E Ratio']}")

    # Include Book Value if non-zero, handling commas
    if float(str(company["Book Value"]).replace(",", "")) != 0:
        chunk_parts.append(f"with a book value of {company['Book Value']}")

    # Include PBV if non-zero, handling commas
    if float(str(company["PBV"]).replace(",", "")) != 0:
        chunk_parts.append(f"and a price-to-book value of {company['PBV']}")

    # Include Market Capitalization unconditionally since no missing values
    chunk_parts.append(f"with a market capitalization of {company['Market Capitalization']}")

    # Join parts into a single sentence
    chunk = ", ".join(chunk_parts) + "."
    chunks.append(chunk)
    metadata.append({
        "source": FUNDAMENTAL_CSV_PATH,
        "company_symbol": company["Company Symbol"],
        "company_name": company["Company Name"],
        "sector": company["Sector"],
        "eps": company["EPS"],
        "pe_ratio": company["P/E Ratio"],
        "book_value": company["Book Value"],
        "pbv": company["PBV"],
        "market_cap": company["Market Capitalization"],
        "row_index": companies.index(company)  # 0-based indexing
    })

print("\nSample chunk content:")
print(chunks[0])
print("\nCorresponding metadata:")
print(metadata[0])

Creating Company Chunks: 100%|██████████| 527/527 [00:00<00:00, 51706.43it/s]


Sample chunk content:
Siddhartha Investment Growth Scheme 3 (SIGS3) (SIGS3), a Mutual Fund company, with an EPS of 0.00 (FY:081-082, Q:10), with a market capitalization of 797,742,000.00.

Corresponding metadata:
{'source': '/content/fundamental_data.csv', 'company_symbol': 'SIGS3', 'company_name': 'Siddhartha Investment Growth Scheme 3 (SIGS3)', 'sector': 'Mutual Fund', 'eps': '0.00 (FY:081-082, Q:10)', 'pe_ratio': '0', 'book_value': '0', 'pbv': '0', 'market_cap': '797,742,000.00', 'row_index': 0}


In [12]:
# Step 5: Set Up Embeddings Model
sentence_transformer = HuggingFaceEmbeddings(
    model_name='sentence-transformers/paraphrase-MiniLM-L3-v2',
    model_kwargs={'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu')}
)

# Step 6: Convert Chunks to Document Objects
documents = [
    Document(
        page_content=chunk,
        metadata=meta
    )
    for chunk, meta in zip(chunks, metadata)
]

<ipython-input-12-62486a8ade8e>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  sentence_transformer = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
from google.colab import files
ZIP_FILE = '/content/fundamental_vector_db.zip'
# Step 7: Create and Save FAISS Vector Store
print("\nCreating FAISS vector store...")
vector_db = FAISS.from_documents(documents, sentence_transformer)
vector_db.save_local(OUTPUT_DIR)
print(f"Fundamental data has been successfully converted to a FAISS vector store and saved at '{OUTPUT_DIR}'.")

# Step 8: Zip the Vector Store Directory
print("\nZipping the fundamental_vector_db folder...")
shutil.make_archive('/content/fundamental_vector_db', 'zip', OUTPUT_DIR)
print(f"Created zip file at '{ZIP_FILE}'.")

# Step 9: Download the Zipped File
print("\nDownloading the zipped fundamental_vector_db...")
files.download(ZIP_FILE)

# Optional: Test the Vector Store Before Downloading
docsearch = FAISS.load_local(
    OUTPUT_DIR,
    sentence_transformer,
    allow_dangerous_deserialization=True
)

query = "Which companies have a high P/E ratio?"
results = docsearch.similarity_search(query, k=5)

print("\nSearch results for query: 'Which companies have a high P/E ratio?'")
for result in results:
    print(f"Company: {result.metadata['company_name']} ({result.metadata['company_symbol']})")
    print(f"Details: {result.page_content}")
    print(f"Source: {result.metadata['source']}, Row Index: {result.metadata['row_index']}")
    print("-" * 50)


Creating FAISS vector store...
Fundamental data has been successfully converted to a FAISS vector store and saved at '/content/fundamental_data_vector_db'.

Zipping the fundamental_vector_db folder...
Created zip file at '/content/fundamental_vector_db.zip'.



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Search results for query: 'Which companies have a high P/E ratio?'
Company: Pokhara Finance Ltd. (PFL) (PFL)
Details: Pokhara Finance Ltd. (PFL) (PFL), a Finance company, with an EPS of -117.86 (FY:081-082, Q:2), and a P/E ratio of -3.83, with a book value of 53.97, and a price-to-book value of 8.36, with a market capitalization of 4,882,330,288.55.
Source: /content/fundamental_data.csv, Row Index: 26
--------------------------------------------------
Company: Shree Investment Finance Co. Ltd. (SIFC) (SIFC)
Details: Shree Investment Finance Co. Ltd. (SIFC) (SIFC), a Finance company, with an EPS of 2.83 (FY:081-082, Q:2), and a P/E ratio of 189.05, with a book value of 136.13, and a price-to-book value of 3.93, with a market capitalization of 5,350,000,000.00.
Source: /content/fundamental_data.csv, Row Index: 471
--------------------------------------------------
Company: Muktinath Krishi Company Limited (MKCL) (MKCL)
Details: Muktinath Krishi Company Limited (MKCL) (MKCL), a Others co

In [14]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L3-v2")

# Check the dimension
embedding_vector = embedding_model.embed_query("test")
print("Embedding dimension:", len(embedding_vector))


Embedding dimension: 384
